In [1]:
import os
import nltk
import pandas as pd
import numpy as np
curentdirectory='/content/drive/MyDrive/Distilbert_model/'
os.chdir(curentdirectory)

In [2]:
# !unzip wordnet.zip
# !unzip stopwords.zip
!pip install torch==1.7.1
!pip install transformers==3.5.1

In [11]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
import torch
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import torch.nn.functional as nnf

In [4]:
DEVICE = "cuda"
MAX_LEN = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10 
BERT_PATH = "distilbert-base-uncased"
MODEL_PATH = "DistilBert_modelh.h5"
TRAINING_FILE = "Cleaned_Training_Testing_Mapping_Files/train_iris_fullcleaned.xlsx"
TOKENIZER = DistilBertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

In [6]:
df=pd.read_excel(TRAINING_FILE)
num_classes=len(df.intent1.unique())

df=pd.read_excel('Cleaned_Training_Testing_Mapping_Files/train_iris_fullcleaned.xlsx')
num_classes=len(df.intent1.unique())
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
MODEL = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_classes)
MODEL.compile(loss=loss,optimizer=optimizer, metrics=[metric])
MODEL.load_weights(MODEL_PATH)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [7]:
import time
import torch.nn as nn
import pandas as pd
tokenizer = TOKENIZER
max_len = MAX_LEN

In [8]:
sent_csv=pd.read_excel('Cleaned_Training_Testing_Mapping_Files/test_iris_fullcleaned.xlsx')
sentences=sent_csv.utterance.values
d= dict([(i,a) for i, a in zip(df.intent1, df.intent)])

In [9]:
maxindx,predictions,maxindx1,maxindx2,maxindx3,maxindx4,predictions1,predictions2,predictions3,predictions4=[],[],[],[],[],[],[],[],[],[]
top_p1,top_p2,top_p3,top_class1,top_class2,top_class3=[],[],[],[],[],[]

In [12]:
for sent in sentences:
    DistilBert_inp=tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids=DistilBert_inp['input_ids']
    attention_masks=DistilBert_inp['attention_mask']
    input_ids=np.asarray(input_ids).reshape(-1,64)
    attention_masks=np.array(attention_masks).reshape(-1,64)
    outputs = MODEL.predict([input_ids,attention_masks],batch_size=32)
    maxidx = outputs[0].argmax(axis=1)
    maxindx.append(int(maxidx))
    print(d[int(maxidx)])

    predictions.append(d[int(maxidx)])
    # outputs = torch.sigmoid(outputs).cpu().detach().numpy()
    print(str(outputs[0]))
    
    prob = nnf.softmax(torch.tensor(outputs[0]),dim=1)
    top_p, top_class = prob.topk(3, dim = 1)
    print("confidence score",top_p)
    print("confidence score1",top_p[0][0])
    
    top_p1.append(float(top_p[0][0]))
    top_p2.append(float(top_p[0][1]))
    top_p3.append(float(top_p[0][2]))
    print("top_class",top_class)
    print("top_class1",top_class[0][0])

    top_class1.append(int(top_class[0][0]))
    top_class2.append(int(top_class[0][1]))
    top_class3.append(int(top_class[0][2]))
    
    predictions1.append(d[int(top_class[0][0])])
    predictions2.append(d[int(top_class[0][1])])
    predictions3.append(d[int(top_class[0][2])])

sent_csv["pred_idx"]=pd.DataFrame(maxindx)
sent_csv["pred_values"]=pd.DataFrame(predictions)
sent_csv["top_p1"]=pd.DataFrame(top_p1)
sent_csv["top_p2"]=pd.DataFrame(top_p2)
sent_csv["top_p3"]=pd.DataFrame(top_p3)
sent_csv["top_class1"]=pd.DataFrame(top_class1)
sent_csv["top_class2"]=pd.DataFrame(top_class2)
sent_csv["top_class3"]=pd.DataFrame(top_class3)
sent_csv["predictions1"]=pd.DataFrame(predictions1)
sent_csv["predictions2"]=pd.DataFrame(predictions2)
sent_csv["predictions3"]=pd.DataFrame(predictions3)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Streaming output truncated to the last 5000 lines.
  -12.107748  -10.893772  -10.223234   -6.9915195  -9.52724    -7.0125923
   -6.0478306  -8.93904   -16.630636  -10.939663   -8.738989  -13.645615
  -11.654695  -12.49904   -11.781734   -8.889314   -9.671102  -14.51406
  -14.524228  -10.515316  -16.211618   -8.264291   -5.1804967 -11.075967
  -16.143784  -12.159372   -8.978119  -14.015563   -8.456766  -10.168627
   -9.797419   -8.494624  -15.538229   -7.520567  -14.25187   -15.718648
  -10.51986   -12.286379   -8.346883  -12.4024105 -10.3050165 -15.560006
  -11.873237  -11.955149  -13.328479  -10.3757105 -13.423626  -10.817192
   -8.869536  -10.444139  -12.5495615  -7.9572897  -9.832301  -10.296014
  -13.22061   -11.199697   -6.855536  -13.649097  -10.577215   -8.703139
  -12.009689   -9.294899  -10.63519    -8.591629   -7.9526052  -6.4564013
   -7.343557  -10.912854   -7.047266   -4.8152914  -8.418881   -6.2075343
   -8.027849   -4.1909175 -13.955972  -15.057946  -10.115921   -8.29795

In [13]:
sent_csv.to_csv('IRIS_predictions.csv',index=None)

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(sent_csv['intent1'],sent_csv['pred_idx'])

0.8115693012600229